In [ ]:
!pip install torchinfo

In [ ]:
import torch
import torchinfo
import cv2
from ultralytics import YOLO


In [ ]:
# load custom yolov5 model
# model = torch.hub.load('ultralytics/yolov8', "custom", path='../ml_models/model_100epochs_v1.pt', force_reload=True).autoshape()
checkpoint = YOLO('../ml_models/model_100epochs_v1.pt')
model = checkpoint.model
# show model information
torchinfo.summary(checkpoint.model, input_size=(1, 3, 640, 640), col_names=("input_size", "output_size", "num_params", "kernel_size", "mult_adds"), depth=4)
# print(checkpoint)

In [ ]:
# draw boxes around detected objects
def analyze_image(model, frame):
    results = model(frame)

    label, coord = results.xyxy[0][:, -1], results.xyxy[0][:, :-1]

    x_shape, y_shape = frame.shape[1], frame.shape[0]
    print("len label", len(label))
    if (len(label) > 0):
        row = coord[0]
        x1, y1, x2, y2 = int(row[0] * x_shape), int(row[1] * y_shape), int(row[2] * x_shape), int(row[3] * y_shape)
        bgr = (0, 255, 0)
        cv2.rectangle(frame, (x1, y1), (x2, y2), bgr, 2)
        print(label, coord)
    # for i in range(len(label)):
    #     row = coord[i]
    #     x1, y1, x2, y2 = int(row[0] * x_shape), int(row[1] * y_shape), int(row[2] * x_shape), int(row[3] * y_shape)
    #     bgr = (0, 255, 0)
    #     cv2.rectangle(frame, (x1, y1), (x2, y2), bgr, 2)
    #     print(label,coord)

    cv2.imshow('frame', frame)


In [ ]:
cap = cv2.VideoCapture('../data/video.mp4')
while True:
    ret, frame = cap.read()
    frame = cv2.resize(frame, (640, 640))
    analyze_image(model, frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()